In [1]:
import gymnasium as gym
import numpy as np
import pickle
import os

def train_q_learning(
    map_name="4x4",
    episodes=30000,
    learning_rate=0.9,
    discount_factor=0.99,
    epsilon=1.0,
    epsilon_decay=0.0002,
    q_file_name="frozen_lake_qtable.pkl",
    slippery=False
):
    # Crear el entorno con o sin resbalones
    env = gym.make('FrozenLake-v1', map_name=map_name, is_slippery=slippery)

    # Reiniciar Q-table cada vez
    q_table = np.zeros((env.observation_space.n, env.action_space.n))

    rng = np.random.default_rng()

    for ep in range(episodes):
        state = env.reset()[0]
        terminated = False
        truncated = False

        while not terminated and not truncated:
            if rng.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            new_state, reward, terminated, truncated, _ = env.step(action)

            q_table[state, action] += learning_rate * (
                reward + discount_factor * np.max(q_table[new_state, :]) - q_table[state, action]
            )
            state = new_state

        # Disminuir epsilon pero no dejar que llegue a cero
        epsilon = max(epsilon - epsilon_decay, 0.01)

    env.close()

    # Guardar la Q-table en archivo
    with open(q_file_name, "wb") as f:
        pickle.dump(q_table, f)

    print(f"✅ Entrenamiento terminado y Q-table guardada en '{q_file_name}'.")

def run_trained_agent(
    map_name="4x4",
    episodes=5,
    q_file_name="frozen_lake_qtable.pkl",
    slippery=False
):
    if not os.path.exists(q_file_name):
        print("❌ Archivo de Q-table no encontrado. Entrena primero.")
        return

    # Cargar entorno con render y condición de resbalón
    env = gym.make('FrozenLake-v1', map_name=map_name, is_slippery=slippery, render_mode='human')

    with open(q_file_name, "rb") as f:
        q_table = pickle.load(f)

    total_wins = 0

    for ep in range(episodes):
        print(f"\n--- Episodio de prueba {ep+1}/{episodes} ---")
        state = env.reset()[0]
        terminated = False
        truncated = False

        while not terminated and not truncated:
            action = np.argmax(q_table[state, :])
            new_state, reward, terminated, truncated, _ = env.step(action)
            state = new_state

            if terminated or truncated:
                if reward == 1:
                    print("✅ ¡El agente llegó a la meta!")
                    total_wins += 1
                else:
                    print("❌ El agente falló o cayó en un hoyo.")
                break

    env.close()
    print(f"\n🎯 Total de episodios ganados: {total_wins}/{episodes} = {100 * total_wins / episodes:.2f}%")

# --- PRUEBA MANUAL ---
if __name__ == "__main__":
    # Entrenamiento
    train_q_learning(
        map_name="4x4",
        episodes=10000,
        learning_rate=0.5,
        discount_factor=0.9,
        epsilon=1.0,
        epsilon_decay=0.0001,
        q_file_name="frozen_lake_qtable.pkl",
        slippery=True  # ← Prueba activando o desactivando esto
    )

    # Evaluación
    run_trained_agent(
        map_name="4x4",
        episodes=5,
        q_file_name="frozen_lake_qtable.pkl",
        slippery=True  # ← Este valor debe coincidir con el del entrenamiento
    )


✅ Entrenamiento terminado y Q-table guardada en 'frozen_lake_qtable.pkl'.

--- Episodio de prueba 1/5 ---
✅ ¡El agente llegó a la meta!

--- Episodio de prueba 2/5 ---
✅ ¡El agente llegó a la meta!

--- Episodio de prueba 3/5 ---
✅ ¡El agente llegó a la meta!

--- Episodio de prueba 4/5 ---
✅ ¡El agente llegó a la meta!

--- Episodio de prueba 5/5 ---
✅ ¡El agente llegó a la meta!

🎯 Total de episodios ganados: 5/5 = 100.00%
